# Introduction au NEL - approche supervisée

Utilisation du module Entity Linked de la bibliothèque Spacy. Exemple par Sofie Van Landeghem (Spacy) adapté au Spacy v3 et traduit au français.

In [1]:
!pip install spacy==3.0.6
!pip install spacy-lookups-data
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart

Application d'un modèle anglais pré-entraîné sur un échantillon de texte

In [2]:
import spacy
nlp = spacy.load("en_core_web_lg")
text = "Tennis champion Emerson was expected to win Wimbledon."
doc = nlp(text)
for ent in doc.ents:
    print(f"Named Entity '{ent.text}' with label '{ent.label_}'")

Named Entity 'Emerson' with label 'PERSON'
Named Entity 'Wimbledon' with label 'EVENT'


Nous constatons que cette phrase contient une personne appelée "Emerson" et un événement appelé "Wimbledon".

Malheureusement, il peut y avoir de nombreuses personnes dans le monde qui s'appellent "Emerson", et ce résultat ne nous dit toujours pas de laquelle il s'agit exactement.


Dans ce cas précis, la phrase nous donne des indices importants : Emerson est manifestement un joueur de tennis professionnel.

En effectuant une recherche sur l'internet, nous pouvons établir que cette phrase parle très probablement de Roy Emerson, un joueur de tennis australien. Nous pouvons à présent associer à cette entité de la phrase son identifiant unique WikiData.
Ses identifiants uniques commencent toujours par un Q, et "Roy Emerson" a l'identifiant Q312545 : https://www.wikidata.org/wiki/Q312545


Pour mettre en œuvre un pipeline Entity Linker, nous avons besoin de trois étapes différentes.

# Creation de la base de connaissance (KB)

La première étape consiste à créer une KB contenant les identifiants uniques des entités qui nous intéressent.

La KB stocke des vecteurs d'entités pré-entraînés. Ces vecteurs sont des versions condensées des descriptions des entités. Des embeddings plus importants permettent de capturer plus d'informations, mais nécessitent également plus de stockage.

Dans ce tutoriel, nous en créerons une KB très simple avec seulement 3 entrées. Nous chargeons les données à partir d'un fichier CSV prédéfini.

Pour cela, il faut copier le fichier entites.csv dans l'espace de travail Google Colaboratory.


In [3]:
import csv
from pathlib import Path

def load_entities():
    entities_loc = Path.cwd().parent / "content" / "entities.csv"  # distributed alongside this notebook

    names = dict()
    descriptions = dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            qid = row[0]
            name = row[1]
            desc = row[2]
            names[qid] = name
            descriptions[qid] = desc
    return names, descriptions

In [4]:
name_dict, desc_dict = load_entities()
for QID in name_dict.keys():
    print(f"{QID}, name={name_dict[QID]}, desc={desc_dict[QID]}")

Q312545, name=Roy Stanley Emerson, desc=Australian tennis player
Q48226, name=Ralph Waldo Emerson, desc=American philosopher, essayist, and poet
Q215952, name=Emerson Ferreira da Rosa, desc=Brazilian footballer


Nous avons ici 3 entrées, de 3 personnes différentes appelées Emerson. Un joueur de tennis australien, un écrivain américain et un footballeur brésilien. Nous utiliserons ces informations pour créer notre base de connaissances. Nous devons définir une dimensionnalité fixe pour les vecteurs d'entités, qui sera 300-D dans notre cas.

In [5]:
from spacy.kb import InMemoryLookupKB
vocab = nlp.vocab
kb = InMemoryLookupKB(vocab=vocab, entity_vector_length=300)

Pour ajouter chaque entrée à la KB, nous encodons sa description en utilisant les vecteurs de mots de notre modèle `nlp`. L'attribut `vector` d'un document est la moyenne de ses vecteurs de mots. Nous devons également fournir une fréquence, qui est un compte brut du nombre de fois qu'une certaine entité apparaît dans un corpus annoté. Dans ce tutoriel, nous n'utilisons pas ces fréquences, donc nous les fixons à une valeur arbitraire.

In [6]:
for qid, desc in desc_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=qid, entity_vector=desc_enc, freq=342)   # 342 is an arbitrary value here

Nous voulons maintenant spécifier des alias ou des synonymes. Nous commençons par ajouter les noms complets. Ici, nous sommes sûrs à 100 % qu'ils se résolvent à leur QID correspondant, puisqu'il n'y a pas d'ambiguïté.

In [7]:
for qid, name in name_dict.items():
    kb.add_alias(alias=name, entities=[qid], probabilities=[1])   # 100% prior probability P(entity|alias)

Nous voulons également ajouter le pseudonyme "Emerson". Nous supposerons que chacun de nos trois Emerson est également célèbre et nous fixerons donc des probabilités égales pour chaque entité.

In [8]:
qids = name_dict.keys()
probs = [0.3 for qid in qids]
kb.add_alias(alias="Emerson", entities=qids, probabilities=probs)  # sum([probs]) should be <= 1 !

4831166512461469197

Ce sera donc notre base de connaissances. Nous pouvons vérifier les entités et les alias qu'elle contient :

In [9]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

Entities in the KB: ['Q215952', 'Q312545', 'Q48226']
Aliases in the KB: ['Roy Stanley Emerson', 'Emerson Ferreira da Rosa', 'Ralph Waldo Emerson', 'Emerson']


Nous pouvons également imprimer les candidats générés pour le nom complet de Roy Emerson, ainsi que pour la mention "Emerson" ou pour toute autre mention aléatoire, comme "Charles".

In [10]:
print(f"Candidates for 'Roy Stanley Emerson': {[c.entity_ for c in kb.get_alias_candidates('Roy Stanley Emerson')]}")
print(f"Candidates for 'Emerson': {[c.entity_ for c in kb.get_alias_candidates('Emerson')]}")
print(f"Candidates for 'Charles': {[c.entity_ for c in kb.get_alias_candidates('Sofie')]}")

Candidates for 'Roy Stanley Emerson': ['Q312545']
Candidates for 'Emerson': ['Q312545', 'Q48226', 'Q215952']
Candidates for 'Charles': []


Nous remarquons que l'interrogation de la KB avec l'alias "Emerson" nous donne 3 candidats, mais si nous l'interrogeons avec un terme inconnu, nous obtenons une liste vide.

Nous pouvons sauvegarder la base de connaissances en appelant la fonction `to_disk` avec un emplacement de sortie.

In [11]:
# change the directory and file names to whatever you like
import os
output_dir = Path.cwd().parent / "content" / "my_output"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
kb.to_disk(output_dir / "my_kb")

Nous pouvons stocker l'objet `nlp` dans un fichier en appelant aussi `to_disk`.

In [12]:
nlp.to_disk(output_dir / "my_nlp")

# Creation du jeu d'entrainement

Nous pouvons stocker l'objet `nlp` dans un fichier en appelant `to_disk` également. Maintenant, nous devons créer des données annotées (ici en JSONL) pour entraîner un algorithme de liaison d'entités.

In [13]:
import json
from pathlib import Path

json_loc = Path.cwd().parent / "content" / "emerson_annotated_text.jsonl" # distributed alongside this notebook
with json_loc.open("r", encoding="utf8") as jsonfile:
    line = jsonfile.readline()
    print(line)   # print just the first line

{"text":"Interestingly, Emerson is one of only five tennis players all-time to win multiple slam sets in two disciplines, only matched by Frank Sedgman, Margaret Court, Martina Navratilova and Serena Williams.","_input_hash":2024197919,"_task_hash":-1926469210,"spans":[{"start":15,"end":22,"text":"Emerson","rank":0,"label":"ORG","score":1,"source":"en_core_web_lg","input_hash":2024197919}],"meta":{"score":1},"options":[{"id":"Q48226","html":"<a href='https://www.wikidata.org/wiki/Q48226'>Q48226: American philosopher, essayist, and poet</a>"},{"id":"Q215952","html":"<a href='https://www.wikidata.org/wiki/Q215952'>Q215952: Brazilian footballer</a>"},{"id":"Q312545","html":"<a href='https://www.wikidata.org/wiki/Q312545'>Q312545: Australian tennis player</a>"},{"id":"NIL_otherLink","text":"Link not in options"},{"id":"NIL_ambiguous","text":"Need more context"}],"_session_id":null,"_view_id":"choice","accept":["Q312545"],"answer":"accept"}



Nous voyons que le texte complet de la phrase originale est stocké, ainsi que de nombreux détails sur la tâche d'annotation. La partie la plus importante est stockée avec la clé `accept` à la fin : c'est la valeur de notre annotation manuelle. Pour cette phrase spécifique et cette mention spécifique, l'option avec la clé `Q312545` a été sélectionnée manuellement. C'est sur cette information que nous allons entraîner notre éditeur de liens d'entités.

# Entrainer l'Entity Linker de Spacy

Pour alimenter notre Entity Linker en données d'entraînement, nous formatons nos données sous la forme d'un tuple structuré. La première partie est le texte brut, et la seconde partie est un dictionnaire d'annotations. Ce dictionnaire définit les entités nommées que nous voulons lier ("entités"), ainsi que les liens de référence ("liens").

In [14]:
import json
from pathlib import Path

dataset = []
json_loc = Path.cwd().parent / "content" / "emerson_annotated_text.jsonl"
with json_loc.open("r", encoding="utf8") as jsonfile:
    for line in jsonfile:
        example = json.loads(line)
        text = example["text"]
        if example["answer"] == "accept":
            QID = example["accept"][0]
            offset = (example["spans"][0]["start"], example["spans"][0]["end"])
            entity_label = example["spans"][0]["label"]
            entities = [(offset[0], offset[1], entity_label)]
            links_dict = {QID: 1.0}
        dataset.append((text, {"links": {offset: links_dict}, "entities": entities}))

Pour vérifier si la conversion est correcte, il suffit d'imprimer le premier échantillon de notre ensemble de données.

In [15]:
dataset[0]

('Interestingly, Emerson is one of only five tennis players all-time to win multiple slam sets in two disciplines, only matched by Frank Sedgman, Margaret Court, Martina Navratilova and Serena Williams.',
 {'links': {(15, 22): {'Q312545': 1.0}}, 'entities': [(15, 22, 'ORG')]})

Nous pouvons également vérifier certaines statistiques dans cet ensemble de données. Combien de cas de chaque QID avons-nous annotés ?

In [16]:
gold_ids = []
for text, annot in dataset:
    for span, links_dict in annot["links"].items():
        for link, value in links_dict.items():
            if value:
                gold_ids.append(link)

from collections import Counter
print(Counter(gold_ids))

Counter({'Q312545': 10, 'Q48226': 10, 'Q215952': 10})


Nous avons obtenu exactement 10 phrases annotées pour chacun de nos Emerson. Parmi ces phrases, nous allons maintenant mettre de côté 6 cas dans un ensemble de test séparé.

In [17]:
import random

train_dataset = []
test_dataset = []
for QID in qids:
    indices = [i for i, j in enumerate(gold_ids) if j == QID]
    train_dataset.extend(dataset[index] for index in indices[0:8])  # first 8 in training
    test_dataset.extend(dataset[index] for index in indices[8:10])  # last 2 in test

random.shuffle(train_dataset)
random.shuffle(test_dataset)

Avec nos ensembles de données correctement configurés, nous allons maintenant créer des objets `Exemple` pour alimenter le processus de formation. Essentiellement, il contient un document avec des prédictions (`predicted`) et un autre avec des annotations gold-standard (`reference`). Au cours de l'apprentissage, le pipeline comparera ses prédictions au gold-standard et mettra à jour les poids du réseau neuronal en conséquence.

Pour le NEL, l'algorithme a besoin d'accéder à des phrases du jeu de validation, car les algorithmes utilisent le contexte de la phrase pour effectuer la désambiguïsation. Vous pouvez soit fournir des annotations `sent_starts` du jeu de validation, soit exécuter un composant tel que `parser` ou `sentencizer` sur vos documents de référence :

For entity linking, the algorithm needs access to gold-standard sentences, because the algorithms use the context from the sentence to perform the disambiguation. You can either provide gold-standard `sent_starts` annotations, or run a component such as the `parser` or `sentencizer` on your reference documents:

In [18]:
from spacy.training import Example

TRAIN_EXAMPLES = []
if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")
sentencizer = nlp.get_pipe("sentencizer")
for text, annotation in train_dataset:
    example = Example.from_dict(nlp.make_doc(text), annotation)
    example.reference = sentencizer(example.reference)
    TRAIN_EXAMPLES.append(example)


Ensuite, nous allons créer un nouveau composant Entity Linking et l'ajouter au pipeline.

Nous devons également nous assurer que le composant `entity_linker` est correctement initialisé. Pour ce faire, nous avons besoin d'une fonction `get_examples` qui retourne des données d'entraînement, ainsi qu'un argument `kb_loader`. Il s'agit d'une fonction "appelable" qui crée la `KnowledgeBase` à partir d'une certaine instance de `Vocab`. Ici, nous allons charger notre KB depuis le disque, en utilisant la fonction intégrée [`spacy.KBFromFile.v1`](https://spacy.io/api/architectures#KBFromFile), qui est définie dans `spacy.ml.models`.

In [19]:
from spacy.ml.models import load_kb

entity_linker = nlp.add_pipe("entity_linker", config={"incl_prior": False}, last=True)
entity_linker.initialize(get_examples=lambda: TRAIN_EXAMPLES, kb_loader=load_kb(output_dir / "my_kb"))

Ensuite, nous exécuterons la boucle d'apprentissage proprement dite pour le nouveau composant, en veillant à n'entraîner que l'entity linker et non les autres composants.

In [20]:
from spacy.util import minibatch, compounding

with nlp.select_pipes(enable=["entity_linker"]):   # train only the entity_linker
    optimizer = nlp.resume_training()
    for itn in range(500):   # 500 iterations takes about a minute to train
        random.shuffle(TRAIN_EXAMPLES)
        batches = minibatch(TRAIN_EXAMPLES, size=compounding(4.0, 32.0, 1.001))  # increasing batch sizes
        losses = {}
        for batch in batches:
            nlp.update(
                batch,
                drop=0.2,      # prevent overfitting
                losses=losses,
                sgd=optimizer,
            )
        if itn % 50 == 0:
            print(itn, "Losses", losses)   # print the training loss
print(itn, "Losses", losses)

0 Losses {'entity_linker': 5.335945904254913}
50 Losses {'entity_linker': 0.6727053821086884}
100 Losses {'entity_linker': 0.7969951430956523}
150 Losses {'entity_linker': 0.9766097664833069}
200 Losses {'entity_linker': 0.6734015047550201}
250 Losses {'entity_linker': 0.47187170386314387}
300 Losses {'entity_linker': 0.8068462312221527}
350 Losses {'entity_linker': 0.22556909918785095}
400 Losses {'entity_linker': 0.9543093045552571}
450 Losses {'entity_linker': 0.9101343353589375}
499 Losses {'entity_linker': 0.48602598905563354}


La valeur Loss (fonction de perte) de la dernière boucle d'apprentissage est assez faible, ce qui est bon signe. Mais pour vraiment vérifier si notre modèle se généralise bien, nous devons le tester sur des données inédites.


# Tester l'Entity Linker

Appliquons-le d'abord à notre phrase originale. Pour chaque entité, nous imprimons le texte et l'étiquette comme précédemment, mais aussi le QID désambiguïsé tel que prédit par notre entity linker.

In [21]:
text = "Tennis champion Emerson was expected to win Wimbledon."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_, ent.kb_id_)

Emerson PERSON Q215952
Wimbledon EVENT NIL


Nous voyons qu'Emerson est désambiguïsé en Q312545, qui est l'identifiant correct du joueur de tennis. Notez également que l'entité "Wimbledon" reçoit l'annotation `NIL`, qui est essentiellement une valeur de remplacement, montrant que le composant NEL n'a pas pu trouver d'identifiant pertinent pour cette entité. Cela s'explique par le fait que notre base de connaissances et le composant Entity Linking n'ont été entrainés que sur des exemples "Emerson", et sont donc assez limités.

Voyons ce que le modèle prédit pour les 6 phrases de notre ensemble de données de test, qui n'ont jamais été vues pendant l'entrainement.

In [22]:
for text, true_annot in test_dataset:
    print(text)
    print(f"Gold annotation: {true_annot}")
    doc = nlp(text)  # to make this more efficient, you can use nlp.pipe() just once for all the texts
    for ent in doc.ents:
        if ent.text == "Emerson":
            print(f"Prediction: {ent.text}, {ent.label_}, {ent.kb_id_}")
    print()

Emerson scored his second international goal on 31 March 1999, in a friendly match against Japan in Tokyo, which Brazil won 2-0.
Gold annotation: {'links': {(0, 7): {'Q215952': 1.0}}, 'entities': [(0, 7, 'ORG')]}
Prediction: Emerson, ORG, Q215952

Emerson was inducted into the International Tennis Hall of Fame in 1982 and the Sport Australia Hall of Fame in 1986.
Gold annotation: {'links': {(0, 7): {'Q312545': 1.0}}, 'entities': [(0, 7, 'ORG')]}
Prediction: Emerson, ORG, Q215952

Carlyle in particular was a strong influence on him; Emerson would later serve as an unofficial literary agent in the United States for Carlyle, and in March 1835, he tried to persuade Carlyle to come to America to lecture.
Gold annotation: {'links': {(53, 60): {'Q48226': 1.0}}, 'entities': [(53, 60, 'ORG')]}
Prediction: Emerson, ORG, Q215952

Emerson made his Brazil debut on 10 September 1997, in a home friendly match against Ecuador, in Salvador, Bahia, also scoring a goal in the match, as Brazil went on to 

Ces résultats peuvent varier légèrement d'un cycle à l'autre, mais en général, le pipeline EL obtient 5 prédictions correctes sur 6 (83 % de précision). Une supposition aléatoire n'aurait permis d'obtenir que 33 %.